In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [3]:
relevant_columns = ['Edema', 'Atelectasis', 'Pneumonia', 'Lung Opacity', 'Consolidation']
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 classes=relevant_columns,
                                                 train_labels = "filtered_dependencies.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 10},
                                                 crop = {"C1": False},
                                                 split_seed = 6122156, )

Chestxray_BCE_E10_B32_C1_N5 could not be created
Chestxray_WBCE_E10_B32_C1_N5 could not be created
Chestxray_CWBCE_E10_B32_C1_N5 could not be created


In [4]:
chexpert_benchmarks.keys()

dict_keys(['BCE_E10_B32_C1_N5', 'WBCE_E10_B32_C1_N5', 'CWBCE_E10_B32_C1_N5'])

In [12]:
densenet_model = simple_architecture_experiment(chexpert_benchmarks["WBCE_E10_B32_C1_N5"], DenseNet121, relevant_columns)

In [13]:
densenet_model.run()

Epoch 1/10
   1/1911 [..............................] - ETA: 0s - loss: 2.2775 - auc: 0.4753 - precision: 0.3162 - recall: 0.7872 - f2_score: 0.6066 - binary_accuracy: 0.4375 - learning rate: 0.0010WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1911/1911 [==============================] - ETA: 0s - loss: 1.8157 - auc: 0.5779 - precision: 0.8394 - recall: 0.6466 - f2_score: 0.6778 - binary_accuracy: 0.8554 - learning rate: 0.0010INFO:tensorflow:Summary name epoch_learning rate is illegal; using epoch_learning_rate instead.

Epoch 00001: val_loss improved from inf to 0.44335, saving model to models/DenseNet121_Chexpert_WBCE_E10_B32_C1_N5/weights.01-0.44.hdf5
1911/1911 [==============================] - 374s 195ms/step - loss: 1.8157 - auc: 0.5779 - precision: 0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


19047/19047 [==============================] - 324s 17ms/step - loss: 0.4047 - auc: 0.6291 - precision: 0.6912 - recall: 0.7259 - f2_score: 0.7187 - binary_accuracy: 0.8185


(<tensorflow.python.keras.callbacks.History at 0x7f6bb4056c18>,
 {'report': '               precision    recall  f1-score   support\n\n        Edema       0.27      0.23      0.25      5201\n  Atelectasis       0.15      0.37      0.21      2747\n    Pneumonia       0.02      0.00      0.00       788\n Lung Opacity       0.91      1.00      0.96     17423\nConsolidation       0.00      0.00      0.00      2717\n\n    micro avg       0.65      0.68      0.66     28876\n    macro avg       0.27      0.32      0.28     28876\n weighted avg       0.62      0.68      0.64     28876\n  samples avg       0.70      0.75      0.68     28876\n',
  'metrics': {'loss': 0.404695987701416,
   'auc': 0.6291096806526184,
   'precision': 0.6912118792533875,
   'recall': 0.725896954536438,
   'f2_score': 0.7186842560768127,
   'binary_accuracy': 0.8184902667999268},
  'predictions': array([[0.13025315, 0.5439868 , 0.10194399, 0.90975463, 0.12342714],
         [0.83223206, 0.14842801, 0.01202178, 0.93335

In [19]:
from src.metrics.metrics import F2Score

dependencies = {
    'F2Score': F2Score
}
pretrained_model = tf.keras.models.load_model('models/DenseNet121_Chexpert_WBCE_E10_B32_C1_N5/DenseNet121_Chexpert_WBCE_E10_B32_C1_N5_20200720-151611.h5', custom_objects=dependencies)

In [ ]:
# freeze all layers except last one
for layer in pretrained_model.layers[:429]:
    layer.trainable = False
for layer in pretrained_model.layers:
    print(layer, layer.trainable)

In [22]:
unfiltered_chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 classes=relevant_columns,
                                                 # train_labels = "filtered_dependencies.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 10},
                                                 crop = {"C1": False},
                                                 split_seed = 6122156, )
experiment = Experiment(unfiltered_chexpert_benchmarks["WBCE_E10_B32_C1_N5"], pretrained_model, model_name='pretrained_DN_dependency')

Chestxray_BCE_E10_B32_C1_N5 could not be created
Chestxray_WBCE_E10_B32_C1_N5 could not be created
Chestxray_CWBCE_E10_B32_C1_N5 could not be created


AttributeError: 'Functional' object has no attribute 'simple_name'